# Installation
## Cytomine:
Using pip, you can do it with these two lines:
~~~
$ curl -s https://packagecloud.io/install/repositories/cytomine-uliege/Cytomine-python-client/script.python.sh | bash
$ pip install cytomine-python-client
~~~


# Import


In [ ]:
from cytomine import Cytomine
from cytomine.models import ProjectCollection, ImageInstanceCollection, AnnotationCollection
import getpass
import shapely
from shapely import wkt
from shapely.affinity import affine_transform
import cv2                             # OpenCV
from imgTools import display, multiDisplay
import os
import logging

# Connect to Cytomine

In [ ]:
host = 'https://learn.cytomine.be'
public_key = getpass.getpass('Public key : ')   # u can get it on your account
private_key = getpass.getpass('Private key : ') # u can get it on your account

conn = Cytomine.connect(host, public_key, private_key, verbose=logging.ERROR)
# Check https://docs.python.org/3.6/library/logging.html#logging-levels  to see other verbose level
print(conn.current_user)

# Example Cytomine

## Get projects

In [ ]:
projects = ProjectCollection().fetch()
print("\nThe projects ids are : \n--------")
for project in projects:
    print(project.id)

## Get images from projects

In [ ]:
projects = ProjectCollection().fetch()
# => Fetch the collection of image instances, filtered by the given project.
image_instances = ImageInstanceCollection().fetch_with_filter("project", projects[0].id)
print(image_instances)
print(f"\nThe images of project {projects[0].id} are : \n--------")

for image in image_instances[0:2]:
    # Every element in the collection is an ImageInstance object.
    # See ImageInstance class for all available properties (width, height, resolution, ...)
    print("Image ID: {} | Width: {} | Height: {} | Resolution: {} | Magnification: {} | Filename: {}".format(
        image.id, image.width, image.height, image.resolution, image.magnification, image.filename
    ))
print("...")
print(f"\nThe fields of an image object are : \n--------")
print(image_instances[0].__dict__)

## Get annotations

In [ ]:
def get_by_id(haystack, needle):
    return next((item for item in haystack if item.id == needle), None)

# We want all annotations in a given project.
annotations = AnnotationCollection()
annotations.project = projects[0].id  # Add a filter: only annotations from this project
# You could add other filters:
# annotations.image = id_image => Add a filter: only annotations from this image
# annotations.images = [id1, id2] => Add a filter: only annotations from these images
# annotations.user = id_user => Add a filter: only annotations from this user
# ...
annotations.showWKT = True  # Ask to return WKT location (geometry) in the response
annotations.showMeta = True  # Ask to return meta information (id, ...) in the response
annotations.showGIS = True  # Ask to return GIS information (perimeter, area, ...) in the response
# ...
# => Fetch annotations from the server with the given filters.
annotations.fetch()
print(annotations)

for i in range(2):
    annotation = annotations[i]
    print(f"\nAnnotation {i}:\n---------")
    print("ID: {} | Image: {} | Project: {} | Term: {} | User: {} | Area: {} | Perimeter: {} | WKT: {}".format(
        annotation.id,
        annotation.image,
        annotation.project,
        annotation.term,
        annotation.user,
        annotation.area,
        annotation.perimeter,
        annotation.location
    ))

    # Annotation location is the annotation geometry in WKT format.
    # See https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry

    # You can use Shapely library to read geometry in WKT format. See https://shapely.readthedocs.io/en/latest/
    # See 'shapely.wkt.loads(wkt)' function in Shapely library.
    geometry = wkt.loads(annotation.location)
    print("Geometry from Shapely: {}".format(geometry))

    # In Cytomine, geometries are referenced using a cartesian coordinate system !
    # See 'shapely.affinity.affine_transform(geom, matrix)' function in Shapely library if needed
    if True:
        # In OpenCV, the y-axis is reversed for points. (https://stackoverflow.com/a/47510950)
        # x' = ax + by + x_off => x' = x
        # y' = dx + ey + y_off => y' = -y + image.height
        # matrix = [a, b, d, e, x_off, y_off]
        image = get_by_id(image_instances, annotation.image)
        geometry_opencv = affine_transform(geometry, [1, 0, 0, -1, 0, image.height])
        print("Geometry with OpenCV coordinate system: {}".format(geometry_opencv))
print("\n...")

## Display annotation on image

In [ ]:
# get image
image_instances = ImageInstanceCollection().fetch_with_filter("project", projects[0].id)
image_id = image_instances[0].id
img = cv2.imread(f'images_database/Team04/{image_instances[0].filename}')
img_1 = img.copy()

# get annotation
annotations = AnnotationCollection()
annotations.showWKT = True  # Ask to return WKT location (geometry) in the response
annotations.showMeta = True  # Ask to return meta information (id, ...) in the response
annotations.showGIS = True  # Ask to return GIS information (perimeter, area, ...) in the response
annotations.image = image_id
annotations.fetch()
for annotation in annotations:
    geometry = wkt.loads(annotation.location)

    image = get_by_id(image_instances, annotation.image)
    geometry_opencv = affine_transform(geometry, [1, 0, 0, -1, 0, image.height])
    pts = geometry_opencv.coords
    if isinstance(geometry_opencv,shapely.geometry.linestring.LineString):
        cv2.line(img, (round(pts[0][0]), round(pts[0][1])), (round(pts[1][0]), round(pts[1][1])), (0, 0, 255), 3)
    else:
        print('NOT ok')
multiDisplay([image_instances[0].filename, image_instances[0].filename], [img_1, img], 2, width=15, height=15)

# Get all images and resp. annotations
The variables:
    - lines_img : list with all the lines images (correctly labeled) in a numpy array
    - lines_ann : list of list of annotation for each image of list_img

In [ ]:
# Dict with img name and id
imgs_dict_id = {}
imgs_dict_height = {}
projects = ProjectCollection().fetch()
for project in projects:
    image_instances = ImageInstanceCollection().fetch_with_filter("project", project.id)
    for image in image_instances:
        imgs_dict_id[image.filename] = image.id
        imgs_dict_height[image.filename] = image.height

In [ ]:
# Lines images
p = "images_database"
all_names = os.listdir(p+"/Example")+os.listdir(p+"/eyes")+os.listdir(p+"/NoEllipses")+os.listdir(p+"/Team01")+\
            os.listdir(p+"/Team02")+os.listdir(p+"/Team03")+os.listdir(p+"/Team04")+os.listdir(p+"/Team05")+\
            os.listdir(p+"/Team06")+os.listdir(p+"/Team07")+os.listdir(p+"/Team08")+os.listdir(p+"/Team09")+\
            os.listdir(p+"/Team10")
lines_img_names = sorted([img for img in all_names if (not "elps" in img) and img.endswith(".png")])

print("Loading the line images...")
lines_img = []
for name in lines_img_names:
    lines_img.append(cv2.imread(p+"/"+name))
    
print("Loading the line annotations...")
lines_ann = []
for name in lines_img_names:
    annotations = AnnotationCollection()
    annotations.showWKT = True  # Ask to return WKT location (geometry) in the response
    annotations.showMeta = True  # Ask to return meta information (id, ...) in the response
    annotations.showGIS = True  # Ask to return GIS information (perimeter, area, ...) in the response
    if name in imgs_dict_id:
        annotations.image = imgs_dict_id[name]
    else:
        print(f"Image {name} not found.")
    annotations.fetch()
    img_ann = []
    err = False
    for annotation in annotations:
        geometry = wkt.loads(annotation.location)
        geometry_opencv = affine_transform(geometry, [1, 0, 0, -1, 0, imgs_dict_height[name]])
        try:
            pts = geometry_opencv.coords
            img_ann.append([pts[0][0], pts[0][1], pts[1][0], pts[1][1]])
        except Exception as e: # The lines were not labeled but the ellipses
            err = True
            print(f"Error with {name} : {e}.")
    if err and len(img_ann) == 0:
        print(f"Error with {name} : removed from lines_img.")
        lines_img.remove(cv2.imread(p+"/"+name))
    else:
        lines_ann.append(img_ann)

In [ ]:
# Ellipses images
p = "images_database"
all_names = os.listdir(p+"/Example")+os.listdir(p+"/eyes")+os.listdir(p+"/NoEllipses")+os.listdir(p+"/Team01")+\
            os.listdir(p+"/Team02")+os.listdir(p+"/Team03")+os.listdir(p+"/Team04")+os.listdir(p+"/Team05")+\
            os.listdir(p+"/Team06")+os.listdir(p+"/Team07")+os.listdir(p+"/Team08")+os.listdir(p+"/Team09")+\
            os.listdir(p+"/Team10")
elps_img_names = sorted([img for img in all_names if ("elps" in img) and img.endswith(".png")])

print("Loading the elps images...")
elps_img = []
for name in elps_img_names:
    elps_img.append(cv2.imread(p+"/"+name))
    
print("Loading the elps annotations...")
# TODO this is very long, there should be as faster way to do it with less fetching
elps_ann = []
for name in elps_img_names:
    annotations = AnnotationCollection()
    annotations.showWKT = True  # Ask to return WKT location (geometry) in the response
    annotations.showMeta = True  # Ask to return meta information (id, ...) in the response
    annotations.showGIS = True  # Ask to return GIS information (perimeter, area, ...) in the response
    if name in imgs_dict_id:
        annotations.image = imgs_dict_id[name]
        annotations.fetch()
        img_ann = []
        err = False
        for annotation in annotations:
            geometry = wkt.loads(annotation.location)
            geometry_opencv = affine_transform(geometry, [1, 0, 0, -1, 0, imgs_dict_height[name]])
            try:
                img_ann.append(geometry_opencv) # TODO change the way the info is stored?
            except Exception as e:
                err = True
                print(f"Error with {name} : {e}.")
        if err and len(img_ann) == 0:
            print(f"Error with {name} : removed from elps_img.")
            elps_img.remove(cv2.imread(p+"/"+name))
        else:
            elps_ann.append(img_ann)
    else: # image not labeled (eg. eye closed images)
        elps_ann.append([])
        print(f"Image {name} not found, empty list added as notation.")
    